In [22]:
import os
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
from keras.utils import vis_utils

In [28]:
# Original width: 600
IM_WIDTH = 224

# Original height: 450
IM_HEIGHT = 168

BATCH_SIZE = 16
SEED = 8348

np.random.seed(SEED)

# global_mean = np.load('../Rahul/Utils/np_mean_rounded_nonweighted.npz')['image']
# global_std = np.load('../Rahul/Utils/np_std_rounded_nonweighted.npz')['image']

In [32]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight

img_gen = ImageDataGenerator(rescale=1./255)

train_gen = img_gen.flow_from_directory('D:\\ISIC2018\\output\\output\\', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['DF', 'VASC', 'AKIEC', 'BCC', 'MEL', 'BKL', 'NV'])
train_gen.class_indices

Found 10015 images belonging to 7 classes.


{'DF': 0, 'VASC': 1, 'AKIEC': 2, 'BCC': 3, 'MEL': 4, 'BKL': 5, 'NV': 6}

In [30]:
print(train_gen.next())

(array([[[[0.8470589 , 0.5529412 , 0.5411765 ],
         [0.83921576, 0.5568628 , 0.56078434],
         [0.8705883 , 0.58431375, 0.6117647 ],
         ...,
         [0.8941177 , 0.6666667 , 0.6627451 ],
         [0.87843144, 0.6627451 , 0.6509804 ],
         [0.8745099 , 0.6627451 , 0.6627451 ]],

        [[0.8431373 , 0.5568628 , 0.58431375],
         [0.8431373 , 0.56078434, 0.5764706 ],
         [0.854902  , 0.5764706 , 0.6       ],
         ...,
         [0.8980393 , 0.6627451 , 0.6627451 ],
         [0.8862746 , 0.654902  , 0.67058825],
         [0.8862746 , 0.65882355, 0.6431373 ]],

        [[0.854902  , 0.58431375, 0.6039216 ],
         [0.8588236 , 0.5882353 , 0.6       ],
         [0.85098046, 0.5921569 , 0.6       ],
         ...,
         [0.90196085, 0.6627451 , 0.6901961 ],
         [0.8941177 , 0.6745098 , 0.6784314 ],
         [0.87843144, 0.65882355, 0.654902  ]],

        ...,

        [[0.81568635, 0.6       , 0.5882353 ],
         [0.8352942 , 0.62352943, 0.58431375

In [11]:
from keras.applications.inception_v3 import InceptionV3
from keras import Model
from keras.layers import (Input, Dense, Activation, Dropout, Conv2D,
                          BatchNormalization, Flatten, 
                          GlobalAveragePooling2D)
from keras.layers import Multiply, Concatenate

inc = InceptionV3(include_top=False, weights='imagenet', input_shape=(IM_HEIGHT, IM_WIDTH, 3))

In [5]:
# from keras.utils.vis_utils import plot_model
# plot_model(m, to_file='inception.png', show_shapes=True, show_layer_names=True)

In [15]:
from keras.layers import Lambda

def split(x):
    return x[:, -1]

def get_output(x):
    return x[:, :2]

# SplitForDecision = Lambda(split)
# SplitForOutput  = Lambda(get_output)

In [63]:
from keras.layers import MaxPooling2D, Concatenate

inc_layers = inc.layers
maxpool_count = 0

for index, layer in enumerate(inc.layers):
    if type(layer) is MaxPooling2D:
        layer.name = 'maxpool_{}'.format(maxpool_count)
        maxpool_count += 1
        print('MaxPool2D Name changed')

print('-------------------------------------')
        
branch_outputs = []
branch_layers = {
    'maxpool_0': 3, 
    'maxpool_1': 1,
    'mixed0': 1, 
    'mixed3': 3
}

for branch_layer, num_outputs in branch_layers.items():
    branch_point = inc.get_layer(branch_layer)
    gap = GlobalAveragePooling2D()(branch_point.output)
    if num_outputs == 1:
        activation = 'sigmoid'
        op = Dense(num_outputs, activation=activation)(gap)
        decision_unit = op
    elif num_outputs > 1:
        activation = 'softmax'
        branch_op = Dense(num_outputs, activation=activation)(gap)
        decision_unit = Lambda(split)(branch_op)
        op = Lambda(get_output)(branch_op)
    branch_outputs.append(op)
    
    print('Branch {} created'.format(branch_layer))

final_op = inc.output
gap_1 = GlobalAveragePooling2D()(final_op)
d_1 = Dense(32)(gap_1)
bn = BatchNormalization(name='unique_name')(d_1)
a_1 = Activation('relu')(bn)
x = Dense(1, activation='sigmoid')(a_1)
branch_outputs.append(x)

print('-------------------------------------')

final_model = Model(inputs=inc.input, outputs=branch_outputs)

for i, layer in enumerate(final_model.layers):
    if type(layer) is BatchNormalization:
        input_shape = layer.input_shape
        gamma = np.ones((input_shape[-1],))
        beta = np.zeros((input_shape[-1],))
        moving_mean = np.zeros((input_shape[-1],))
        moving_variance = np.ones((input_shape[-1],))
        config = layer.get_config()
        if config['scale'] and config['center']:
            layer.set_weights([gamma, beta, moving_mean, moving_variance])
        elif config['scale'] and not config['center']:
            layer.set_weights([gamma, moving_mean, moving_variance])
        elif not config['scale'] and config['center']:
            layer.set_weights([beta, moving_mean, moving_variance])
        print('Weights Changed')
# final_model.get_layer('unique_name').set_weights([gamma, beta, moving_mean, moving_variance])


print('Model built')

MaxPool2D Name changed
MaxPool2D Name changed
MaxPool2D Name changed
MaxPool2D Name changed
-------------------------------------
Branch maxpool_0 created
Branch maxpool_1 created
Branch mixed0 created
Branch mixed3 created
-------------------------------------
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
Weights Changed
We

In [66]:
for i, layer in enumerate(final_model.layers):
    if type(layer) is BatchNormalization:
        print(layer.get_weights())
        print(layer.get_config())
    if i > 100: break

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)]
{'name': 'batch_normalization_287', 'trainable': True, 'axis': 3, 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': False, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}}, 'beta_regularizer': None, 'gamma_regularizer': None, 'beta_constraint': None, 'gamma_constraint': None}
[array([0., 0

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.,

In [21]:
from keras.utils.vis_utils import plot_model
plot_model(final_model, to_file='custom_inception_v2.png', show_shapes=True, show_layer_names=True)

In [ ]:
main_input = Input((5, 5, 2))
# x = Conv2D(5, (3, 3), padding='valid')(main_input)
x = main_input
decision_unit = Flatten()(x)
decision_unit = Dense(1, activation='sigmoid')(decision_unit)

level_2 = Multiply()([x, decision_unit])

#level_2 = Conv2D(3, (3, 3), padding='valid')([x, decision_unit])
level_2_gap = GlobalAveragePooling2D()(level_2)
out = Dense(1, activation='sigmoid')(level_2_gap)
out=Concatenate(axis=-1)([decision_unit, out])
model = Model(inputs=main_input, outputs=[level_2, decision_unit, out])
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
a = np.random.randn(3, 5, 5, 2)
pred = model.predict(a)
print(a)
print('_________')
print(pred[0])
print('___')
print(pred[2])

In [ ]:
print(np.sum((a*0.8707708)-pred[0])**2)

In [ ]:
from keras.optimizers import Adam
optimizer - Adam()
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer=optimizer)
model.fit_generator(train_gen, 
                    steps_per_epochs=train_gen.samples//BATCH_SIZE + 1, 
                    validation_data=val_gen, 
                    validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                    shuffle=False, 
                    epochs=1)